In [1]:
import pickle
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm # 颜色映射
import numpy as np
import os
import argparse

# 获取颜色的辅助函数 (类似你的 ipynb 文件中的)
def get_color(id, num_vehicles):
    """根据车辆ID和总车辆数获取颜色"""
    if num_vehicles <= 10:
        cmap = cm.get_cmap("tab10") # 使用tab10颜色映射（最多10种颜色）
        return cmap(id % 10)
    elif num_vehicles <= 20:
        cmap = cm.get_cmap("tab20") # 使用tab20颜色映射（最多20种颜色）
        return cmap(id % 20)
    else:
        # 如果车辆更多，使用matplotlib默认颜色循环
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        return colors[id % len(colors)]

def visualize(output_base_dir):
    """
    加载单个评估运行的数据并进行可视化。

    Args:
        output_base_dir (str): eval.py 创建的主输出目录
                                (例如, 'results/single_run_vis').
    """
    # --- 定义文件路径 ---
    # 假设第一个（也是唯一一个）实例的数据在 batch0-sample0 子目录中
    instance_data_dir = os.path.join(output_base_dir, "batch0-sample0")
    summary_file = os.path.join(output_base_dir, "summary.json")
    history_file = os.path.join(instance_data_dir, "history_data.pkl")
    route_file = os.path.join(instance_data_dir, "route_info.pkl")

    # --- 检查文件是否存在 ---
    if not os.path.exists(summary_file):
        print(f"错误: 汇总文件未找到于 {summary_file}")
        return
    if not os.path.exists(history_file):
        print(f"错误: 历史文件未找到于 {history_file}。请确认 state.py 中的 SAVE_HISTORY 是否为 True 并重新运行 eval.py。")
        return
    if not os.path.exists(route_file):
        print(f"错误: 路线信息文件未找到于 {route_file}。请确认运行 eval.py 时是否使用了 --visualize_routes。")
        return

    print("正在加载数据...")
    # --- 加载数据 ---
    try:
        with open(summary_file, 'r') as f:
            summary_data = json.load(f)
            # 如果 eval_batch_size=1, avg_obj 就是这个实例的目标值
            objective_value = summary_data.get("avg_obj", "N/A")
            # 你可能也想加载其他指标，例如 avg_tour_length
    except Exception as e:
        print(f"加载汇总文件 {summary_file} 时出错: {e}")
        return

    try:
        with open(history_file, 'rb') as f:
            history_data = pickle.load(f)
            time_points = history_data.get("time", [])
            veh_batt_history = history_data.get("veh_batt", []) # 这是个列表的列表 [[veh0_hist], [veh1_hist], ...]
            # loc_batt_history = history_data.get("loc_batt", []) # 如果需要基站电量，取消注释
            # down_loc_history = history_data.get("down_loc", []) # 如果需要断电基站数量，取消注释
    except Exception as e:
        print(f"加载历史文件 {history_file} 时出错: {e}")
        return

    try:
        with open(route_file, 'rb') as f:
            route_data = pickle.load(f)
            loc_coords = np.array(route_data["loc_coords"]) # 基站/客户点坐标
            depot_coords = np.array(route_data["depot_coords"]) # 充电站坐标
            routes = route_data["route"] # 列表的列表, [[veh0_route], [veh1_route], ...] route是节点ID序列
            ignored_depots = np.array(route_data.get("ignored_depots", [])) # 优雅地处理可能缺失的键
    except Exception as e:
        print(f"加载路线文件 {route_file} 时出错: {e}")
        return

    # 检查加载的数据是否有效
    if not time_points or not veh_batt_history or not routes:
         print("错误: 加载的数据缺少必要的历史或路线信息。")
         return

    num_vehicles = len(routes)
    num_locs = len(loc_coords)
    # 合并所有坐标，方便索引
    coords = np.concatenate([loc_coords, depot_coords], axis=0)

    print("数据加载成功。正在生成图像...")

    # --- 创建图像 ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7)) # 1行2列的子图布局
    # 添加总标题，包含目标值
    fig.suptitle(f"EVRP 单实例可视化 (目标值: {objective_value:.4f})", fontsize=16)

    # --- 图 1: EV 电池电量历史 ---
    ax1.set_title("EV 电池电量随时间变化")
    ax1.set_xlabel("时间 (h)")
    ax1.set_ylabel("电池电量 (单位?)") # 你需要确认单位是kWh还是归一化的
    ax1.grid(True, linestyle='--', alpha=0.6)
    if time_points:
        ax1.set_xlim(0, max(time_points) * 1.05 if time_points else 1) # x轴范围加5%边距

    min_batt, max_batt = float('inf'), float('-inf') # 用于自动调整y轴范围
    for veh_id in range(num_vehicles):
        # 检查是否有该车辆的历史数据，并且历史数据不为空
        if veh_id < len(veh_batt_history) and veh_batt_history[veh_id]:
            # 检查时间点数量和电量历史记录数量是否匹配
            hist_len = len(veh_batt_history[veh_id])
            time_len = len(time_points)
            plot_len = min(hist_len, time_len) # 取较短的长度进行绘制
            if hist_len != time_len:
                 print(f"警告: 车辆 {veh_id} 的电量历史记录数 ({hist_len}) 与时间点数 ({time_len}) 不匹配。将绘制前 {plot_len} 个点。")

            if plot_len > 0:
                batt_levels = veh_batt_history[veh_id][:plot_len]
                ax1.plot(time_points[:plot_len], batt_levels,
                         label=f"车辆 {veh_id}", color=get_color(veh_id, num_vehicles), alpha=0.8, linewidth=1.5)
                # 更新电量范围
                min_b = min(batt_levels)
                max_b = max(batt_levels)
                if min_b < min_batt: min_batt = min_b
                if max_b > max_batt: max_batt = max_b
        else:
            print(f"警告: 未找到车辆 {veh_id} 的电池历史数据。")

    # 设置y轴范围，如果找到了有效的电量数据
    if max_batt > min_batt:
        ax1.set_ylim(min_batt * 0.95, max_batt * 1.05) # y轴范围加5%边距
    ax1.legend(fontsize='small')

    # --- 图 2: 路线地图 ---
    ax2.set_title("车辆路线网络")
    markersize = 50 # 调整标记大小以便看清

    # 绘制基站/客户点 (黑色圆点)
    if len(loc_coords) > 0:
        ax2.scatter(loc_coords[:, 0], loc_coords[:, 1], s=markersize, c="black", label="客户点", marker="o", zorder=3)
        # 为客户点添加编号
        for i in range(num_locs):
            ax2.text(loc_coords[i, 0] + 0.01, loc_coords[i, 1] + 0.01, str(i), fontsize=8, zorder=4)

    # 绘制充电站 (红色方块)
    # 检查 ignored_depots 是否与 depot_coords 长度匹配
    use_ignore_mask = len(ignored_depots) == len(depot_coords)
    active_depots = depot_coords[~ignored_depots] if use_ignore_mask else depot_coords
    inactive_depots = depot_coords[ignored_depots] if use_ignore_mask else np.empty((0, 2))
    depot_ids = np.arange(num_locs, num_locs + len(depot_coords)) # 全局ID
    active_depot_ids = depot_ids[~ignored_depots] if use_ignore_mask else depot_ids
    inactive_depot_ids = depot_ids[ignored_depots] if use_ignore_mask else np.array([])

    if len(active_depots) > 0:
        ax2.scatter(active_depots[:, 0], active_depots[:, 1], marker="s", s=markersize*1.5, c="red", label="充电站", zorder=3)
        # 为活动的充电站添加编号
        for i, idx in enumerate(active_depot_ids):
             ax2.text(active_depots[i, 0] + 0.01, active_depots[i, 1] + 0.01, str(idx), fontsize=8, zorder=4, color='red')

    # (可选) 绘制被忽略的充电站 (灰色方块)
    if len(inactive_depots) > 0:
         ax2.scatter(inactive_depots[:, 0], inactive_depots[:, 1], marker="s", s=markersize*1.5, facecolors='none', edgecolors='gray', label="忽略的充电站", zorder=2, alpha=0.5)
         # 为忽略的充电站添加编号
         for i, idx in enumerate(inactive_depot_ids):
              ax2.text(inactive_depots[i, 0] + 0.01, inactive_depots[i, 1] + 0.01, str(idx), fontsize=8, zorder=4, color='gray', alpha=0.7)


    # 绘制路线
    for veh_id, route in enumerate(routes):
        if not route: continue # 跳过空路线
        # 获取路线中所有节点的坐标
        route_coords = coords[route]
        color = get_color(veh_id, num_vehicles)
        # 绘制连接节点的线段
        ax2.plot(route_coords[:, 0], route_coords[:, 1], color=color, alpha=0.6, linewidth=1.5, zorder=1, label=f"车辆 {veh_id} 路径" if veh_id==0 else "") # 只给第一条路径加总标签
        # (可选) 绘制箭头指示方向，可能会让图很乱
        for i in range(len(route) - 1):
             st = route_coords[i]
             ed = route_coords[i+1]
             # 检查距离是否为0，避免绘制无效箭头
             if np.linalg.norm(ed - st) > 1e-6:
                 ax2.annotate('', xy=ed, xytext=st,
                              arrowprops=dict(arrowstyle="-|>", color=color, lw=0.7,
                                             mutation_scale=10), # 调整箭头大小
                              zorder=2) # 确保箭头在线段之上
        # 标记每辆车的起始点
        if route: # 确保路线非空
            start_node_id = route[0]
            start_coord = coords[start_node_id]
            ax2.scatter(start_coord[0], start_coord[1], marker='X', s=markersize*2, color=color, edgecolors='black', zorder=5, label=f"车辆 {veh_id} 起点")

    ax2.set_xlabel("X 坐标")
    ax2.set_ylabel("Y 坐标")
    # 将图例放在图外右侧，避免遮挡
    ax2.legend(fontsize='small', loc='center left', bbox_to_anchor=(1.02, 0.5))
    ax2.set_aspect('equal', adjustable='box') # 保持X/Y轴比例一致
    ax2.grid(True, linestyle='--', alpha=0.5)

    # --- 最终调整与保存 ---
    plt.tight_layout(rect=[0, 0, 0.9, 0.95]) # 调整布局，留出图例和标题空间
    # 保存图像
    save_path = os.path.join(output_base_dir, "solution_visualization.png")
    plt.savefig(save_path, dpi=200) # 保存为较高分辨率
    print(f"可视化图像已保存至: {save_path}")
    plt.close(fig) # 关闭图像窗口，释放内存

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="从评估结果可视化EVRP解决方案。")
    parser.add_argument("--output_dir", type=str, required=True,
                        help="eval.py 创建的主输出目录 (例如: results/single_run_vis)")
    args = parser.parse_args()

    visualize(args.output_dir)

usage: ipykernel_launcher.py [-h] --output_dir OUTPUT_DIR
ipykernel_launcher.py: error: the following arguments are required: --output_dir


SystemExit: 2

/Users/dxk/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
